# ÁLGEBRA LINEAR COMPUTACIONAL

## SEGUNDA AVALIAÇÃO - 2020.2 - PROFESSOR ERITO MARQUES   

---
### GRUPO:
**Ícaro Santos Barcelos Pereira - _2017780187_**\
**Leandro Bataglia Pereira - _2018780199_**\
**Pedro Henrique Farolfi Camelo - _2018780326_**

---

Repositório dos algoritmos solicitados na segunda prova da disciplina de Álgebra Linear Computacional.
##### Instruções:
As respostas devem ser enviadas no formato jupyter notebook.\
Os trabalhos devem ser feitos em grupo. Não é permitida troca de informações entre os grupos.\
Fixada a data de entrega, não serão aceitos trabalhos entregues fora dessa data.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore",category=UserWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)

In [ ]:
import numpy as np
import pandas as pd
import math
from normas import *
from criterios import *
from cholesky import *
from iterativos import *
from benford import *
from fatoracaoLU import fatoracaoLU
from substituicao import resolve_substituicao
from sympy import Matrix, init_printing
init_printing()
#BIBLIOTECA QUE NÃO ESTOU USANDO MAS PODE SER ÚTIL PARA EXIBIR AS MATRIZES/VETORES = display(Matrix(*variavel que representa o dado*)

In [ ]:
# Coverti isso aqui em um procedimento, a parada é que vc pode mandar isso
# ocorrer no início de qualquer célula e pode alterar o excel em tempo de execução :)

# CONVERSÃO DOS DADOS EM ARRAYS DA NUMPY COM TYPE 'float64'
def ler_matriz_A_vetor_b(file_excel):
    df = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name="A e b")
    df_matriz = df.iloc[:,0:-2]
    df_vetor = df.iloc[:,-1]
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor = np.ravel(np.array(df_vetor, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor

### 1) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e avalie:

In [ ]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes")

**a)** se A é tridiagonal;

In [ ]:
def verifica_tridiagonal(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> não é tridiagonal")
        return False

    for i in range(linhas):
        for j in range(colunas):
            if np.abs(i-j) <= 1 and matriz_A[i,j] == 0:
                return False
            if np.abs(i-j) >= 2 and matriz_A[i,j] != 0:
                return False
    return True

In [ ]:
verifica_tridiagonal(matriz, debug=True)

**b)** se A é triangular superior;

In [ ]:
def verifica_tri_superior(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> não é triangular superior")
        return False

    for i in range(linhas):
        for j in range(i):
            if matriz_A[i,j] != 0:
                return False
    return True

In [ ]:
verifica_tri_superior(matriz, debug=True)

**c)** se A é triangular inferior;

In [ ]:
def verifica_tri_inferior(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        print("A matriz não é quadrada -> não é triangular inferior")
        return False

    for j in range(colunas):
        for i in range(j):
            if matriz_A[i,j] != 0:
                return False
    return True

In [ ]:
verifica_tri_inferior(matriz)

**d)** se Traço de A é maior que 4.

In [ ]:
def traco(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> traço não está definido.")
        return False
    
    n, traco = min((linhas, colunas)), 0
    for i in range(n):
        traco += matriz_A[i,i]
    if traco > 4:
        print("O traço da matriz é " + str(traco) + ", portanto é maior que 4")
        return True
    print("O traço da matriz é " + str(traco) + ", portanto não é maior que 4")
    return False

In [ ]:
A = np.array([[1,2,3],[1,2,3]])
traco(A, debug=True)

In [ ]:
traco(matriz, debug=True)

---
### 2) Implemente um algoritmo que leia uma matriz A (triangular) e um vetor b (de uma planilha excel) e resolva o sistema Ax = b por substituição para frente ou para trás conforme o caso.

In [ ]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes")

In [ ]:
def resolve_substituicao(matriz_A, vetor_b):
    if checa_tamanho_matriz_vetor(matriz_A, vetor_b) == False:
        return False
        
    if(verifica_tri_inferior(matriz_A)):
        return resolve_subs_frente(matriz_A, vetor_b)

    elif(verifica_tri_superior(matriz_A)):
        return resolve_subs_tras(matriz_A,vetor_b)
        
    else:
        print("A matriz não é triangular")
        return False

def checa_tamanho_matriz_vetor(matriz_A, vetor_b):
    lin_A, col_A = np.shape(matriz_A)
    col_b = np.shape(vetor_b)
    
    try:
        assert lin_A == col_b[0]
    except:
        print("A quantidade de linhas em A e de colunas em b são distintas")
        return False
    return True
    
def resolve_subs_frente(matriz_A, vetor_b): # Resolve um sistema com matriz triangular inferior
    n = len(matriz_A)
    vetor_resultado = np.array(np.copy(vetor_b), dtype='f8')

    for i in range(n):
        for j in range(i):
            vetor_resultado[i] = vetor_resultado[i] - matriz_A[i,j] * vetor_resultado[j]
        vetor_resultado[i] = vetor_resultado[i]/matriz_A[i,i]
    return vetor_resultado

def resolve_subs_tras(matriz_A,vetor_b): # Resolve um sistema com matriz triangular superior
    n = len(matriz_A)
    vetor_resultado = np.array(np.copy(vetor_b), dtype='f8')

    for i in range(n-1,-1,-1):
        for j in range(n-1,i,-1):
            vetor_resultado[i] = vetor_resultado[i] - matriz_A[i,j] * vetor_resultado[j]
        vetor_resultado[i] = vetor_resultado[i]/matriz_A[i,i]
    return vetor_resultado

In [ ]:
print(resolve_substituicao(matriz, vetor))

---
### 3) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:



In [ ]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes")

**a)** Avalie se é possível obter o fator de Cholesky;

In [ ]:
def verifica_pos_def(matriz_A):
    if verifica_inversa(matriz_A) and verifica_simetria(matriz_A):
        n = len(matriz_A)
        for i in range(n):
            soma = 0
            for j in range(n):
                soma += matriz_A[i,j]
            if(soma == 0 or matriz_A[i,i] <= 0):
                print("A matriz não é positiva definida.")
                return False
        print("A matriz é positiva definida.")
        return True
    print("A matriz não é positiva definida.")
    return False

In [ ]:
verifica_pos_def(matriz)

**b)** Resolva o sistema Ax = b usando a fatoração de Cholesky.

In [ ]:
def decomposicao_cholesky(matriz_A, vetor_b):
    if verifica_pos_def(matriz_A):
        return None
    n = len(matriz_A) # n é a ordem da matriz a
    matriz_R = np.copy(np.triu(matriz_A)) # matriz_r é uma cópia triangular superior da matriz A

    for i in range(n):
        for k in range(i): # Não é executado quando i = 0
            matriz_R[i,i] = matriz_R[i,i] - math.pow(matriz_R[k,i],2) # Define os termos da diagonal principal
        if matriz_R[i,i] <= 0: # Se algum elemento da diagonal principal for menor ou igual a zero, por definição, a matriz não é positiva definida
            return None
        matriz_R[i,i] = math.sqrt(matriz_R[i,i]) 
        for j in range(i+1,n): # Não é executado quando i = n
            for k in range(i): # Não é executado quando i = 0
                matriz_R[i,j] = matriz_R[i,j] - (matriz_R[k,i] * matriz_R[k,j])
            matriz_R[i,j] = matriz_R[i,j]/matriz_R[i,i]

    return resolve_cholesky(matriz_R, vetor_b)

def resolve_cholesky(matriz_R, vetor_b):
    matriz_Rt = np.copy(np.transpose(matriz_R)) # matriz_rt é a matriz R transposta
    
    vetor_y = subs.resolve_substituicao(matriz_Rt, vetor_b) # Faz o cálculo de Rt * y = b para obter o vetor y
    vetor_x = subs.resolve_substituicao(matriz_R, vetor_y) # Faz o cálculo de R * x = y para obter o vetor x (resultado)

    return vetor_x # Retorna o vetor x (resultado)

In [ ]:
decomposicao_cholesky(matriz, vetor)

---
### 4) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:

In [ ]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes")

**a)** Avalie se algum critério é satisfeito para aplicação dos métodos de Gauss-Seidel, Jacobi e SOR;

In [ ]:
def criterio_linhas(matriz_A): # Verificar o critério das linhas para o método de Jacobi
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    soma = 0

    for i in range(n): # Percorre toda a matriz
        for j in range(m):
            if i != j: # Desconsidera os termos da diagonal principal
                soma += np.abs(matriz_A[i,j])

        if soma >= np.abs(matriz_A[i,i]): # Verifica se a soma de todos os termos (sem contar o termo da diagonal principal) de cada linha é menor ou maior que o termo da diagonal principal (Critério das linhas)
            return False

        soma = 0
    return True

In [ ]:
def criterio_sassenfeld(matriz_A):
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que auxilia nos somatórios dos termos tanto de Aij*Beta, quanto de Aij
    vetor_betao = np.zeros(m) # Vetor de Betas iniciado com termos iguais a zero

    for i in range(n):
        for j in range(i): # j vai até i-1 (desconsidera a primeira iteração)
            aux += np.abs(matriz_A[i,j]) * vetor_betao[j] # A partir da segunda iteração, inclui a multiplicação de Aij pelos Betas anteriores no somatório (já que na primeira iteração não há Betas anteriores, obviamente)
        for j in range(i,n):
            aux += np.abs(matriz_A[i,j]) # Somatório dos módulos dos termos de Aij
        vetor_betao[i] = aux/np.abs(matriz_A[i,i]) #Divide o valor da soma pelo termo da diagonal principal
        aux = 0 # Zera a variável auxiliar pra próxima iteração
    
    print("Maior betão:", np.max(vetor_betao))
    if np.max(vetor_betao) < 1: # Verifica se a condição do critério de Sassenfeld (max(B) < 1) é satisfeita
        return True
    return False

In [ ]:
criterio_linhas(matriz)

In [ ]:
criterio_sassenfeld(matriz)

**b)** Resolva o sistema Ax = b usando os métodos iterativos de Gauss-Seidel, Jacobi e SOR.

## Jacobi

In [ ]:
def jacobi(matriz_A, vetor_b, vetor_x=0, N=10000, p=1e-15): # Função que executa o algoritmo de Jacobi
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj

    if isinstance(vetor_x, int): # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)
    
    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1

    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função (N) se a condição de parada não for satisfeita
        for i in range(n):
            for j in range(n):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj
            
            vetor_aux[i] = (vetor_b[i] - aux)/matriz_A[i,i] # Executa a iteração do método com (bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        termos_encontrados = 0
        for i in range(n): #Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0:
                if np.abs(vetor_aux[i] - vetor_x[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p 
                    termos_encontrados += 1
            else:
                if np.abs(vetor_aux[i] - vetor_x[i]) <= p*np.abs(vetor_x[i]):
                    termos_encontrados += 1
                    
        if(termos_encontrados == n):
            print("(Jacobi) Resultado encontrado com o critério de parada, na iteração:", k)
            return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(Jacobi) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [ ]:
print(jacobi(matriz, vetor, N=100))

## Gauss-Seidel

In [ ]:
def seidel(matriz_A, vetor_b, vetor_x=0, N=1000, p=1e-15): # Função que executa o algoritmo de Gauss-Seidel
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj

    if isinstance(vetor_x, int): # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)

    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1
    	
    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    if criterio_sassenfeld(matriz_A): # Verifica se o critério de Sassenfeld é satisfeito
        print("O critério de Sassenfeld foi satisfeito.")
    else:
        print("ATENÇÃO: O critério de Sassenfeld não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função
        for i in range(n):
            for j in range(m):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    if i > j:
                        aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
                    else:
                        aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não foram atualizados (de i+1 a n) que ficam no vetor x

            vetor_aux[i] = (vetor_b[i] - aux)/matriz_A[i,i] # Executa a iteração do método com (bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        termos_encontrados = 0
        for i in range(n): #Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0:
                if np.abs(vetor_aux[i] - vetor_x[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p 
                    termos_encontrados += 1
            else:
                if np.abs(vetor_aux[i] - vetor_x[i]) <= p*np.abs(vetor_x[i]):
                    termos_encontrados += 1
                    
        if(termos_encontrados == n):
                print("(Seidel) Resultado encontrado com o critério de parada, na iteração:", k)
                return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(Seidel) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [ ]:
print(seidel(matriz, vetor))

## SOR

In [ ]:
def calcular_omega_otimo(matriz_A):
    if verifica_pos_def(matriz_A) == False or verifica_tridiagonal(matriz_A, debug=True) == False:
        print("Não há como encontrar w ótimo.")
        return None       

    n = len(matriz_A)
    D = np.copy(matriz_A)
    for i in range(n):
        for j in range(n):
            if i != j:
                D[i,j] = 0.

    D = np.linalg.inv(D)
    L = np.tril(matriz_A)
    U = np.triu(matriz_A)

    for i in range(n):
        for j in range(n):
            if i == j:
                L[i,j] = 0.
                U[i,j] = 0.

    LU = L + U
    T = np.matmul(D, LU)
    P = np.linalg.eig(T)
    p = np.amax(P[0])
    
    print("omega: {}".format(2/(1 + math.sqrt(1 - pow(p, 2)))))
    return 2/(1 + math.sqrt(1 - pow(p, 2)))
    

In [ ]:
def sor(matriz_A, vetor_b, vetor_x=0, N=10000, p=1e-15, w=1.7): # Quando w = 1 é igual ao Seidel
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj
    
    omega = calcular_omega_otimo(matriz_A)
    
    if omega != None:
        w = omega

    if isinstance(vetor_x, int): # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)

    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1

    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    if criterio_sassenfeld(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério de Sassenfeld foi satisfeito.")
    else:
        print("ATENÇÃO: O critério de Sassenfeld não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função
        for i in range(n):
            for j in range(m):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    if i > j:
                        aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
                    else:
                        aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não foram atualizados (de i+1 a n) que ficam no vetor x

            vetor_aux[i] = (1-w)*vetor_x[i] + (w*(vetor_b[i] - aux))/matriz_A[i,i] # Executa a iteração do método com (1-w)*xi(bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        termos_encontrados = 0
        for i in range(n): #Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0:
                if np.abs(vetor_aux[i] - vetor_x[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p 
                    termos_encontrados += 1
            else:
                if np.abs(vetor_aux[i] - vetor_x[i]) <= p*np.abs(vetor_x[i]):
                    termos_encontrados += 1
                    
        if(termos_encontrados == n):
                print("(SOR) Resultado encontrado com o critério de parada, na iteração:", k)
                return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(SOR) Resultado encontrado com o número máximo de iterações:")
    return vetor_x # Retorna o vetor solução

In [ ]:
sor(matriz, vetor)

---
### 5) Implemente um algoritmo que leia uma matriz A (de uma planilha excel) e:

In [ ]:
def ler_matriz_A(file_matrix):
    df_matriz = pd.read_excel(r"dados/"+file_matrix+".xlsx", header=None, sheet_name="A") # Le o arquivo do excel que contém a matriz A sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    return matriz

In [ ]:
matriz = ler_matriz_A("prova_ALC_testes")

**a)** Calcule a transposta de A;

In [ ]:
def calcula_transposta(matriz_A):
    if not verifica_quadrada(matriz_A):
        print("A matriz não é quadrada.")
        return None

    n = len(matriz_A)
    matriz_At = np.zeros((n,n), dtype='f8')

    for i in range(n):
        for j in range(n):
            if i == j:
                matriz_At[i,j] = matriz_A[i,j]
            else:
                matriz_At[i,j] = matriz_A[j,i]
    
    return matriz_At

In [ ]:
calcula_transposta(matriz)

**b)** Calcule o determinante de A;

In [ ]:
def eliminacao_gauss(matriz_A):
    n = len(matriz_A)
    Am = np.array(matriz_A).astype("float64")

    for i in range(n):
        for j in range(i+1, n):
            if Am[i,i] == 0:
                Am[i,i] = 1e-18

            x = Am[j,i]/Am[i,i] # x é o multiplicador necessário para executar a eliminação
            for k in range(n):
               Am[j,k] -= Am[i,k] * x # aplica o multiplicador x para zerar os elementos abaixo da diagonal principal

    return calcula_determinante(Am, n) # retorna o vetor resultado

def calcula_determinante(matriz_A, n):
    det = 1
    for i in range(n):
        det *= matriz_A[i,i]
    return det

In [ ]:
eliminacao_gauss(matriz)

**c)** Calcule a matriz adjunta de A;

In [ ]:
def calcula_adjunta(matriz_A):
    if verifica_inversa(matriz_A):
        matriz_Adj = np.linalg.inv(matriz_A) * eliminacao_gauss(matriz_A)
        return matriz_Adj

In [ ]:
print(calcula_adjunta(matriz))

**d)** Calcule o traço de A;

In [ ]:
def traco(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    n, traco = min((linhas, colunas)), 0

    for i in range(n):
        traco += matriz_A[i,i]
    return traco


In [ ]:
print("Traço de A:", traco(matriz))

**e)** Avalie se A é simétrica;

In [ ]:
def verifica_simetria(matriz_A):
    if verifica_quadrada(matriz_A):
        n = len(matriz_A)
        for i in range(n):
            for j in range(n):
                if (i != j) and (matriz_A[i,j] != matriz_A[j,i]):
                    print("A matriz não é simétrica")
                    return False
        print("A matriz é simétrica")
        return True
    print("A matriz não é quadrada")
    return False

In [ ]:
verifica_simetria(matriz)

**f)** Avalie se A é ortogonal;

In [ ]:
def verifica_ortogonal(matriz_A):
    if not verifica_quadrada:
        print("A matriz não é quadrada.")
        return False
    
    if not verifica_inversa:
        print("A matriz é singular.")
        return False
    
    n = len(matriz_A)
    matriz_At = calcula_transposta(matriz_A)
    matriz_Id = np.identity(n)

    matriz_Resultado = np.matmul(matriz_A, matriz_At)

    if (matriz_Resultado-matriz_Id).all() <= 1e-10:
        print("A matriz é ortogonal.")
        return True
    else:
        print("A matriz não é ortogonal.")
        return False

In [ ]:
verifica_ortogonal(matriz)

**g)** Calcule a norma de linha, norma de Frobenius e norma coluna de A.

In [ ]:
def soma_linha(matriz_A): # norma-infinita matricial
    n, max, x = len(matriz_A), 0, 0

    for i in range(n):
        for j in range(n):
            x += math.fabs(matriz_A[i,j])
        
        if x > max:
            max = x

    return max

In [ ]:
soma_linha(matriz)

In [ ]:
def frobenius(matriz_A): # norma-2 matricial
    n, x = len(matriz_A), 0

    for i in range(n):
        for j in range(n):
            x += math.pow(math.fabs(matriz_A[i,j]), 2)

    return x ** (1/2)

In [ ]:
frobenius(matriz)

In [ ]:
def soma_coluna(matriz_A): # norma-1 matricial
    n, max, x = len(matriz_A), 0, 0

    for j in range(n):
        for i in range(n):
            x += math.fabs(matriz_A[i,j])
        
        if x > max:
            max = x
    
    return max

In [ ]:
soma_coluna(matriz)

**h)** Avalie se A é positiva definida.

In [ ]:
def verifica_pos_def(matriz_A):
    if verifica_inversa(matriz_A) and verifica_simetria(matriz_A):
        n = len(matriz_A)
        for i in range(n):
            soma = 0
            for j in range(n):
                soma += matriz_A[i,j]
            if(soma == 0 or matriz_A[i,i] <= 0):
                print("A matriz não é positiva definida.")
                return False
        print("A matriz é positiva definida.")
        return True
    print("A matriz não é positiva definida.")
    return False

In [ ]:
verifica_pos_def(matriz)

---
### 6) Construa um algoritmo que receba dois vetores x e y e uma matriz A positiva definida (de uma planilha excel). Calcule:

In [ ]:
def ler_matriz_A_vetor_x_vetor_y(file_name, file_matrix, file_array_x, file_array_y):
    df_matriz = pd.read_excel(r"dados/"+file_name+".xlsx", header=None, sheet_name=file_matrix) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_vetor_x = pd.read_excel(r"dados/"+file_name+".xlsx", header=None, sheet_name=file_array_x) # Le o arquivo do excel que contém o vetor x sem cabeçalho
    df_vetor_y = pd.read_excel(r"dados/"+file_name+".xlsx", header=None, sheet_name=file_array_y) # Le o arquivo do excel que contém o vetor y sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor_x = np.ravel(np.array(df_vetor_x, dtype='f8')) # Converte o vetor b em um array da numpy
    vetor_y = np.ravel(np.array(df_vetor_y, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor_x, vetor_y


In [ ]:
matriz, vetor_x, vetor_y = ler_matriz_A_vetor_x_vetor_y("prova_ALC_testes(1)", "matriz_A(2)", "vetor_b(1)", "vetor_b(1)")

**a)** Norma 1, norma 2 e norma infinito de x e y.

In [ ]:
def manhattan(vetor): # norma-1 vetorial
    n, x = len(vetor), 0

    for i in range(n):
        x += np.fabs(vetor[i])
    return x

In [ ]:
print(manhattan(vetor_x))
print(manhattan(vetor_y))

In [ ]:
def euclidiana(vetor): # norma-2 vetorial
    n, x = len(vetor), 0

    for i in range(n):
        x += np.fabs(vetor[i]) ** 2

    return x ** (1/2)

In [ ]:
print(euclidiana(vetor_x))
print(euclidiana(vetor_y))

In [ ]:
def infinita(vetor): # norma-infinita vetorial
    n, max = len(vetor), vetor[0]
    
    for i in range(1, n):
        if np.fabs(vetor[i]) > max:
            max = np.fabs(vetor[i])

    return max

In [ ]:
print(infinita(vetor_x))
print(infinita(vetor_y))

**b)** Ângulo entre x e y;

In [ ]:
def produto_interno(vetor_x, vetor_y):
    n, p_interno = len(vetor_x), 0

    for i in range(n):
        p_interno += vetor_x[i] * vetor_y[i]
    
    return p_interno

def angulo_vetores(vetor_x, vetor_y):
    n = len(vetor_x)

    p_interno = produto_interno(vetor_x, vetor_y)
    norma_x = euclidiana(vetor_x)
    norma_y = euclidiana(vetor_y)
    p_normas = norma_x * norma_y

    div = p_interno/p_normas

    return np.arccos(div)

In [ ]:
angulo_vetores(vetor_x, vetor_y)

**c)** Norma de x e y induzida por A;

In [ ]:
def matricial_induzida(matriz_A, vetor_b, norma=manhattan):
    
    if norma == manhattan:
        mat = np.matmul(matriz_A, vetor_b)
        return manhattan(mat)/manhattan(vetor_b)
    
    elif norma == euclidiana:
        mat = np.matmul(matriz_A, vetor_b)
        return euclidiana(mat)/euclidiana(vetor_b)
    
    elif norma == infinita:
        mat = np.matmul(matriz_A, vetor_b)
        return infinita(mat)/infinita(vetor_b)

In [ ]:
matricial_induzida(matriz, vetor_x)
matricial_induzida(matriz, vetor_y)

**d)** Produto interno de x e y.

In [ ]:
def produto_interno(vetor_x, vetor_y):
    n, p_interno = len(vetor_x), 0

    for i in range(n):
        p_interno += vetor_x[i] * vetor_y[i]
    
    return p_interno

In [ ]:
produto_interno(vetor_x, vetor_y)

---
### 7) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel). Encontre a fatoração LU de A.

In [ ]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes")

In [ ]:
fatoracaoLU(matriz,vetor)

---
### 8) Construa um algoritmo que receba uma matriz A positiva (de uma planilha excel). Calcule:

In [ ]:
matriz = ler_matriz_A("prova_ALC_testes")

**a)** O número condição;

**b)** Retorne uma mensagem ao usuário sobre sua interpretação.

In [ ]:
def condition_number(matriz):
    inv = np.linalg.inv(matriz)
    kond = frobenius(matriz) * frobenius(inv)
    print("Número condição: {}".format(kond))
    print("Para um sistema linear com essa matriz, o erro esperado causado por uma perturbação em b vai será de aproximadamente:")
    return kond

In [ ]:
condition_number(matriz)

---
### 9) Construa um algoritmo que receba três matrizes: U, sigma e V (de uma planilha excel) e:

In [ ]:
def ler_matrizes_USV(file_excel):
    df = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name="SVD")
    data = np.array(df)

    n, m = np.shape(data)
    nan_find = ~np.isnan(data).all(axis=0)
    nan_index = []

    for i in range(len(nan_find)):
        if nan_find[i] == False:
            nan_index.append(i)

    data_U = data[:,:nan_index[0]]
    data_S = data[:,nan_index[0]+1:nan_index[1]]
    data_Vt = data[:,nan_index[1]+1:m]

    matriz_U = data_U[~np.isnan(data_U).all(axis=1)]
    matriz_S = data_S[~np.isnan(data_S).all(axis=1)]
    matriz_Vt = data_Vt[~np.isnan(data_Vt).all(axis=1)]

    return matriz_U, matriz_S, matriz_Vt

In [ ]:
matriz_U, matriz_S, matriz_Vt = ler_matrizes_USV("prova_ALC_testes")

**a)** Avalie se o produto das três matrizes representa uma decomposição SVD de alguma matriz X;

**b)** Caso a resposta à pergunta anterior seja negativa, retorne ao usuário uma explicação justificando esse fato.

In [ ]:
# U e V ortogonais e S não-quadrada e diagonal
def verifica_diagonal(matriz):
    n, m = np.shape(matriz)

    for i in range(n):
        for j in range(m):
            if i != j and matriz[i,j] != 0:
                print("A matriz não é diagonal.")
                return False
    print("A matriz é diagonal.")
    return True

def verifica_SVD(matriz_U, matriz_S, matriz_Vt):
    print("Para as matrizes USVt serem a decomposição SVD de uma matriz X:\n-> U e Vt devem ser ortogonais.\n-> S deve ser diagonal não-quadrada.\n")
    print("Verificando se U é ortogonal...")
    if verifica_ortogonal(matriz_U):
        print("\nVerificando se Vt é ortogonal...")
        if verifica_ortogonal(matriz_Vt):
            print("\nVerificando se S é diagonal e não-quadrada...")
            if verifica_diagonal(matriz_S):
                nS, mS = np.shape(matriz_S)
                if nS != mS:
                    print("A matriz é não-quadrada.")
                    print("")
                    mU = len(matriz_U)
                    nVt = len(matriz_Vt)

                    if mU == nS and mS == nVt:
                        print("As matrizes são a decomposição SVD da matriz X abaixo:")
                        matriz_X = np.matmul(np.matmul(matriz_U, matriz_S), matriz_Vt)
                        print(matriz_X)

                        return True
                    else:
                        print("As matrizes não têm dimensões compatíveis para executar um produto de matrizes.")
                        return False
                else:
                    print("A matriz S é quadrada.")
                    return False
            else:
                print("A matriz S não é diagonal.")
                return False
        else:
            print("A matriz Vt não é ortogonal.")
            return False
    else:
        print("A matriz U não é ortogonal.")
        return False

In [ ]:
verifica_SVD(matriz_U, matriz_S, matriz_Vt)

---
### 10) Construa um algoritmo que leia uma planilha em Excel com 4 colunas e calcule a estatística de Benford (para 1 dígito) para cada uma das colunas.

In [ ]:
def ler_dados_benford(file_name): # Le o arquivo do excel que contém os dados para testar o algoritmo de ocorrências (Benford)
    df_benford = pd.read_excel(r"dados/"+file_name+".xlsx", header=None)
    dataset_benford = np.array(df_benford) # Converte a matriz de dados em um array numpy
    return dataset_benford

def benford(dados):
    n, m = np.shape(dados)
    primeiro_digito = np.zeros((n,m)) # Matriz auxiliar para armazenar o primeiro dígito de cada posição da matriz de dados

    # Looping para armazenar os primeiros dígitos dos elementos da matriz de dados na matriz primeiro_digito
    for j in range(m):
        for i in range(n):
            num = [x for x in str(dados[i,j])]
            for value in num:
                if value != '0' and value != '.' and value != '-':
                    primeiro_digito[i,j] = int(value)
                    break

    # Verifica as ocorrências de 1 a 9 na matriz primeiro_digito por colunas
    for i in range(1, 10):
        count = np.count_nonzero(primeiro_digito == i, axis=0)
        # Mostra a porcentagem de ocorrências pra cada ocorrência de 1 a 9 da matriz primeiro_digito
        for j in range(m):
            if j == 0:
                print('Porcentagem de ocorrência de primeiro dígito "'+str(i)+'" em cada coluna:', end=' ')
            print("{:.4f}".format((count[j]/n)*100)+'%', end=' | ')
        print('')

In [ ]:
benford(ler_dados_benford("dados_benford"))